In [37]:
import pymysql
import numpy as np
import pandas as pd
import time

MYSQL_CONN = pymysql.connect(
    host='db-6slkr-kr.vpc-pub-cdb.ntruss.com',
    port=3306,
    user='domain',
    passwd='bookcode7!',
    db='bookcode',
    charset='utf8mb4'
)

def conn_mysqldb():
    if not MYSQL_CONN.open:
        MYSQL_CONN.ping(reconnect=True)
    return MYSQL_CONN

if __name__ == '__main__':
    mysql_db = conn_mysqldb()
    db_cursor = mysql_db.cursor()

    sql = f"SELECT * FROM User WHERE name = '이수빈';"
    
    db_cursor.execute(sql) 

    result = db_cursor.fetchall()
    result = pd.DataFrame(result)
    
    booklist=result[5][0].split('/')
    
    print(booklist)
    

['폭풍의 언덕', '동급생']


In [45]:
from sklearn.metrics.pairwise import cosine_similarity

def str_to_vector(str_vector):    
    str_vector=str_vector.replace('[','').replace(']','').split()
    list_vector = list(map(float, str_vector))
    vector=np.array(list_vector)    
    return vector

def recommend_by_title(title):

    db = conn_mysqldb()
    db_cursor = db.cursor()
    sql = "SELECT title, vector FROM ReviewTest;"
    db_cursor.execute(sql)
    result = db_cursor.fetchall()

    book_df = pd.DataFrame(result,columns=['title','vector'])
    book_df['vector_']=book_df['vector'].apply(str_to_vector)

    embedding_list=list(book_df['vector_'])
    cosine_similarities = cosine_similarity(embedding_list, embedding_list)

    books = book_df[['title']]
    # 책의 제목을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장.
    indices = pd.Series(book_df.index, index = book_df['title']).drop_duplicates() 
    idx = indices[title]

    # 입력된 책과 줄거리(document embedding)가 유사한 책 5개 선정.
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]

    # 가장 유사한 책 5권의 인덱스
    book_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = books.iloc[book_indices].reset_index(drop=True)

    # 데이터프레임으로부터 순차적으로 이미지를 출력
    return_list=list()
    for i in book_indices:
        return_list.append(books.loc[i]['title'])    
    titles = '\n'.join(return_list)
    print(titles)

In [46]:
for book in booklist:
    recommend_by_title(book)

돌이킬 수 있는
죄와 벌 2
한국단편소설 70
동물농장
파리대왕
불안한 사람들
82년생 김지영
카라마조프 가의 형제들 세트
루팡의 딸
진이, 지니
